# Linear / Ridge / Lasso Regression

## Mục tiêu
- Xây dựng mô hình Linear / Ridge / Lasso Regression để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 

## Giới thiệu mô hình

Linear Regression là mô hình hồi quy tuyến tính cổ điển, mô tả mối quan hệ giữa biến đầu vào và đầu ra dưới dạng phương trình tuyến tính.

### **Ưu điểm:**

- Dễ hiểu, dễ triển khai.

- Huấn luyện rất nhanh.

- Phù hợp khi quan hệ gần tuyến tính.

### **Nhược điểm:**

- Không phù hợp cho dữ liệu phi tuyến.

- Dễ bị ảnh hưởng bởi hiện tượng đa cộng tuyến.

- Nhạy cảm với outlier.

## Bước 1 - Import các thư viện cần thiết

### 1.1. Import thư viện

In [25]:
import os
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

RANDOM_STATE = 42

### 1.2. Cấu hình thư mục

In [26]:
os.makedirs("../models/1_linear_regression", exist_ok=True)

## Bước 2 - Đọc dữ liệu đã tiền xử lý
Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [27]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (dữ liệu đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3124, 13)

5 dòng đầu tiên của tập train (dữ liệu đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Denmark,DNK,2017,-0.203264,-0.418953,81.102439,1.682694,-0.047741,1.673510,0.642797,0.754689,0.118599,0.069223
1,"Korea, Dem. People's Rep.",PRK,2017,-0.056076,-0.530921,73.034000,-0.412271,0.028651,-0.245951,-1.294730,0.439985,-0.216848,2.087276
2,Madagascar,MDG,2008,-0.091998,1.007790,61.992000,-0.607170,0.547296,-1.651029,-2.194047,-2.540432,-0.528445,2.542268
3,Greece,GRC,2018,-0.166799,-0.945727,81.787805,0.170491,-0.209157,1.356075,0.642797,0.754689,0.161554,-1.033944
4,South Sudan,SSD,2019,-0.169071,1.000983,58.129000,-0.412271,0.028651,-1.375450,-2.593358,-2.691194,-0.278341,1.279234


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [28]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3124, 10)
Kích thước y_train: (3124,)


## Bước 4 - Xây dựng và huấn luyện mô hình Linear Regression 


### 4.1. Fit mô hình LinearRegression() trên tập train.

In [29]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_lin = linear_model.predict(X_train)

### 4.2. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [30]:
# Tính 3 loại loss: MAE, RMSE và R2
lin_rmse = np.sqrt(mean_squared_error(y_train, y_pred_lin))
lin_r2  = r2_score(y_train, y_pred_lin)
lin_mae = mean_absolute_error(y_train, y_pred_lin)

print("KẾT QUẢ MÔ HÌNH")
print("="*60)
print("Model: LinearRegression:")
print(f"RMSE loss: {lin_rmse:.3f}")
print(f"MAE loss: {lin_mae:.3f}")
print(f"R2 score: {lin_r2:.3f}")

# Lưu mô hình
joblib.dump(linear_model, "../models/1_linear_regression/linear.pkl")

KẾT QUẢ MÔ HÌNH
Model: LinearRegression:
RMSE loss: 4.254
MAE loss: 3.166
R2 score: 0.767


['../models/1_linear_regression/linear.pkl']

## Bước 5 - Xây dựng và huấn luyện mô hình Ridge
### Mục tiêu:
- Tối ưu hóa siêu tham số Ridge bằng RandomizedSearchCV (k=5)


### 5.1. Huấn luyện mô hình sử dụng phương pháp k-fold cross-validation (k=5) 

In [31]:
from scipy.stats import loguniform

param_distributions_ridge = {
    "alpha": loguniform(1e-4, 1e3), 
    "fit_intercept": [True, False],
    "solver": ["auto", "svd", "lsqr", "sag", "saga"],   
    "tol": loguniform(1e-6, 1e-2),    
}

ridge = Ridge(random_state=RANDOM_STATE)
ridge_cv = RandomizedSearchCV(
    ridge, 
    param_distributions_ridge, 
    scoring='neg_mean_squared_error', 
    cv=5, 
    n_jobs=-1, 
    verbose=2,
    n_iter=60
)

# Huấn luyện mô hình
ridge_cv.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END alpha=3.893043207722651, fit_intercept=False, solver=auto, tol=0.000592350100248415; total time=   0.0s
[CV] END alpha=3.893043207722651, fit_intercept=False, solver=auto, tol=0.000592350100248415; total time=   0.0s
[CV] END alpha=3.893043207722651, fit_intercept=False, solver=auto, tol=0.000592350100248415; total time=   0.0s
[CV] END alpha=3.893043207722651, fit_intercept=False, solver=auto, tol=0.000592350100248415; total time=   0.0s
[CV] END alpha=3.893043207722651, fit_intercept=False, solver=auto, tol=0.000592350100248415; total time=   0.0s
[CV] END alpha=133.0991190040917, fit_intercept=True, solver=sag, tol=0.0015733437106388313; total time=   0.0s
[CV] END alpha=133.0991190040917, fit_intercept=True, solver=sag, tol=0.0015733437106388313; total time=   0.0s
[CV] END alpha=133.0991190040917, fit_intercept=True, solver=sag, tol=0.0015733437106388313; total time=   0.0s
[CV] END alpha=133.0991190040917, fit

/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warni

[CV] END alpha=35.75353340896942, fit_intercept=True, solver=sag, tol=0.0002536395957418751; total time=   0.0s
[CV] END alpha=0.35784942776627066, fit_intercept=False, solver=saga, tol=0.0001031855821284009; total time=   1.4s
[CV] END alpha=0.006324401295228044, fit_intercept=False, solver=lsqr, tol=5.938976116513759e-05; total time=   0.0s
[CV] END alpha=0.12234244176278429, fit_intercept=True, solver=sag, tol=0.0013010806723361397; total time=   0.0s
[CV] END alpha=0.8697668125436941, fit_intercept=False, solver=svd, tol=5.1654549143971285e-06; total time=   0.0s
[CV] END alpha=0.35784942776627066, fit_intercept=False, solver=saga, tol=0.0001031855821284009; total time=   1.4s
[CV] END alpha=10.995226118614024, fit_intercept=False, solver=saga, tol=0.0005646185605199857; total time=   1.5s
[CV] END alpha=0.12234244176278429, fit_intercept=True, solver=sag, tol=0.0013010806723361397; total time=   0.0s
[CV] END alpha=0.12234244176278429, fit_intercept=True, solver=sag, tol=0.0013010

/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END alpha=0.02215249877455589, fit_intercept=True, solver=sag, tol=0.004047827560839034; total time=   0.1s
[CV] END alpha=0.02215249877455589, fit_intercept=True, solver=sag, tol=0.004047827560839034; total time=   0.0s
[CV] END alpha=0.021019291288802686, fit_intercept=False, solver=saga, tol=0.00028262737491103015; total time=   1.4s
[CV] END alpha=0.02215249877455589, fit_intercept=True, solver=sag, tol=0.004047827560839034; total time=   0.0s
[CV] END alpha=0.021019291288802686, fit_intercept=False, solver=saga, tol=0.00028262737491103015; total time=   1.4s
[CV] END alpha=0.021019291288802686, fit_intercept=False, solver=saga, tol=0.00028262737491103015; total time=   1.4s


...

[CV] END alpha=18.31412085478862, fit_intercept=False, solver=saga, tol=2.567640419492575e-06; total time=   0.5s
[CV] END alpha=0.020685704810911417, fit_intercept=False, solver=sag, tol=0.000326638292928602; total time=   0.5s
[CV] END alpha=5.0828843915881174, fit_intercept=False, solver=saga, tol=8.697037400080078e-05; total time=   0.4s
[CV] END alpha=0.16606336427775067, fit_intercept=False, solver=sag, tol=0.00020420501810427397; total time=   0.4s
[CV] END alpha=13.79622890757886, fit_intercept=False, solver=saga, tol=0.00020705834106093032; total time=   0.5s
[CV] END alpha=5.0828843915881174, fit_intercept=False, solver=saga, tol=8.697037400080078e-05; total time=   0.4s
[CV] END alpha=5.0828843915881174, fit_intercept=False, solver=saga, tol=8.697037400080078e-05; total time=   0.5s
[CV] END alpha=5.0828843915881174, fit_intercept=False, solver=saga, tol=8.697037400080078e-05; total time=   0.5s
[CV] END alpha=13.79622890757886, fit_intercept=False, solver=saga, tol=0.000207

/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warni

[CV] END alpha=0.9443523746265285, fit_intercept=False, solver=saga, tol=6.336951997230563e-06; total time=   0.3s
[CV] END alpha=0.020685704810911417, fit_intercept=False, solver=sag, tol=0.000326638292928602; total time=   0.2s
[CV] END alpha=9.179037118459458, fit_intercept=False, solver=saga, tol=0.00044388286270802534; total time=   0.2s
[CV] END alpha=13.79622890757886, fit_intercept=False, solver=saga, tol=0.00020705834106093032; total time=   0.2s
[CV] END alpha=0.16606336427775067, fit_intercept=False, solver=sag, tol=0.00020420501810427397; total time=   0.2s
[CV] END alpha=0.020685704810911417, fit_intercept=False, solver=sag, tol=0.000326638292928602; total time=   0.2s
[CV] END alpha=0.9443523746265285, fit_intercept=False, solver=saga, tol=6.336951997230563e-06; total time=   0.2s
[CV] END alpha=9.179037118459458, fit_intercept=False, solver=saga, tol=0.00044388286270802534; total time=   0.2s
[CV] END alpha=13.79622890757886, fit_intercept=False, solver=saga, tol=0.00020

/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/bonxom/miniconda3/envs/courseML/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


,estimator,Ridge(random_state=42)
,param_distributions,"{'alpha': <scipy.stats....x75f594190620>, 'fit_intercept': [True, False], 'solver': ['auto', 'svd', ...], 'tol': <scipy.stats....x75f594247fe0>}"
,n_iter,60
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


### 5.2. Hiển thị siêu tham số tối ưu

In [32]:
best_ridge = ridge_cv.best_estimator_
best_params_svr = ridge_cv.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params_svr.items():
    print(f"  {param:20s}: {value}")

SIÊU THAM SỐ TỐI ƯU
  alpha               : 2.210327620314769
  fit_intercept       : True
  solver              : saga
  tol                 : 0.007960121055564123


### 5.3. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [33]:
# Tính 3 loại loss: MAE, RMSE và R2
y_pred_ridge = best_ridge.predict(X_train)
ridge_rmse = np.sqrt(mean_squared_error(y_train, y_pred_ridge))
ridge_r2  = r2_score(y_train, y_pred_ridge)
ridge_mae = mean_absolute_error(y_train, y_pred_ridge)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Ridge Regression:")
print(f"RMSE loss: {ridge_rmse:.3f}")
print(f"MAE loss: {ridge_mae:.3f}")
print(f"R2 score: {ridge_r2:.3f}")

# Lưu mô hình
joblib.dump(best_ridge, f"../models/1_linear_regression/ridge.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Ridge Regression:
RMSE loss: 4.254
MAE loss: 3.168
R2 score: 0.767


['../models/1_linear_regression/ridge.pkl']

## Bước 6 - Xây dựng và huấn luyện mô hình Lasso
### Mục tiêu:
- Tối ưu hóa siêu tham số Lasso bằng RandomizedSearchCV (k=5)


### 6.1. Huấn luyện mô hình sử dụng phương pháp k-fold cross-validation (k=5) 

In [34]:
param_distributions_lasso = {
    "alpha": loguniform(1e-4, 1e2),   # 0.0001 → 100 (Lasso nhạy hơn)
    "fit_intercept": [True, False],
    "selection": ["cyclic", "random"],   # random thường tốt hơn
    "max_iter": [1000, 2000, 5000, 10000],
    "tol": loguniform(1e-6, 1e-2),
}

lasso = Lasso(random_state=RANDOM_STATE, max_iter=5000)
lasso_cv = RandomizedSearchCV(
    lasso, 
    param_distributions_lasso, 
    scoring='neg_mean_squared_error', 
    cv=5, 
    n_jobs=-1,
    n_iter=60,
    verbose=2
)

lasso_cv.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END alpha=0.00016129321943488242, fit_intercept=False, max_iter=1000, selection=cyclic, tol=1.4418253304584416e-06; total time=   0.0s
[CV] END alpha=0.00016129321943488242, fit_intercept=False, max_iter=1000, selection=cyclic, tol=1.4418253304584416e-06; total time=   0.0s
[CV] END alpha=0.00016129321943488242, fit_intercept=False, max_iter=1000, selection=cyclic, tol=1.4418253304584416e-06; total time=   0.0s
[CV] END alpha=0.00016129321943488242, fit_intercept=False, max_iter=1000, selection=cyclic, tol=1.4418253304584416e-06; total time=   0.0s
[CV] END alpha=0.00016129321943488242, fit_intercept=False, max_iter=1000, selection=cyclic, tol=1.4418253304584416e-06; total time=   0.0s
[CV] END alpha=5.653265102384227, fit_intercept=True, max_iter=1000, selection=random, tol=1.1142341682119565e-05; total time=   0.0s
[CV] END alpha=5.653265102384227, fit_intercept=True, max_iter=1000, selection=random, tol=1.11423416821

,estimator,Lasso(max_ite...ndom_state=42)
,param_distributions,"{'alpha': <scipy.stats....x75f594204e00>, 'fit_intercept': [True, False], 'max_iter': [1000, 2000, ...], 'selection': ['cyclic', 'random'], ...}"
,n_iter,60
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


### 6.2. Hiển thị siêu tham số tối ưu

In [35]:
best_lasso = lasso_cv.best_estimator_
best_params_lasso = lasso_cv.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params_lasso.items():
    print(f"  {param:20s}: {value}")

SIÊU THAM SỐ TỐI ƯU
  alpha               : 0.02740693607554183
  fit_intercept       : True
  max_iter            : 10000
  selection           : cyclic
  tol                 : 6.76588685650463e-06


### 6.3. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [36]:
# Tính 3 loại loss: MAE, RMSE và R2
y_pred_lasso = best_lasso.predict(X_train)
lasso_rmse = np.sqrt(mean_squared_error(y_train, y_pred_lasso))
lasso_r2  = r2_score(y_train, y_pred_lasso)
lasso_mae = mean_absolute_error(y_train, y_pred_lasso)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Lasso Regression:")
print(f"RMSE loss: {lasso_rmse:.3f}")
print(f"MAE loss: {lasso_mae:.3f}")
print(f"R2 score: {lasso_r2:.3f}")

# Lưu mô hình
joblib.dump(best_lasso, f"../models/1_linear_regression/lasso.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Lasso Regression:
RMSE loss: 4.255
MAE loss: 3.167
R2 score: 0.767


['../models/1_linear_regression/lasso.pkl']

## Kết luận
Tổng kết quá trình huấn luyện 3 mô hình (Linear, Ridge, Lasso)
- Đã huấn luyện thành công các mô hình Linear/Ridge/Lasso"
- Phương pháp 5-fold CV được dùng để chọn siêu tham số cho Ridge & Lasso